In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [2]:
%%fenicsx

from mpi4py import MPI
from dolfinx import fem, mesh, io
from dolfinx.fem.petsc import LinearProblem
import ufl
from ufl import Identity, grad, inner, sqrt, sym, tr, dx, dot, TestFunction, TrialFunction
import numpy as np

# Create mesh
msh = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

# Displacement function space (2D vector)
V = fem.functionspace(msh, ("Lagrange", 1, (2,)))

# Material properties
E = 1.0e9      # Young's modulus (Pa)
nu = 0.3       # Poisson's ratio
mu = E / (2.0 * (1.0 + nu))
lmbda = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))

# ============================================
# Step 1: Set boundary conditions
# ============================================

# Fix left boundary (fully fixed at x=0)
def left_boundary(x):
    return np.isclose(x[0], 0.0)

# Find boundary degrees of freedom
left_facets = mesh.locate_entities_boundary(msh, msh.topology.dim - 1, left_boundary)
left_dofs = fem.locate_dofs_topological(V, msh.topology.dim - 1, left_facets)

# Fixed boundary condition (displacement = 0)
bc_left = fem.dirichletbc(np.array([0.0, 0.0], dtype=np.float64), left_dofs, V)

# ============================================
# Step 2: Define loading
# ============================================

# Apply traction on right boundary (at x=1)
def right_boundary(x):
    return np.isclose(x[0], 1.0)

# Measure for boundary integration
right_facets = mesh.locate_entities_boundary(msh, msh.topology.dim - 1, right_boundary)
facet_tag = mesh.meshtags(msh, msh.topology.dim - 1, right_facets,
                          np.full(len(right_facets), 1, dtype=np.int32))

ds = ufl.Measure("ds", domain=msh, subdomain_data=facet_tag)

# Traction vector (pulling to the right)
traction = fem.Constant(msh, np.array([1.0e6, 0.0], dtype=np.float64))  # 1 MPa

# ============================================
# Step 3: Define weak form
# ============================================

def eps(v):
    """Strain tensor"""
    return sym(grad(v))

def sigma(v):
    """Stress tensor"""
    return 2.0 * mu * eps(v) + lmbda * tr(eps(v)) * Identity(len(v))

# Test and trial functions
u = TrialFunction(V)
v = TestFunction(V)

# Weak form: internal virtual work = external virtual work
# ∫ σ(u) : ε(v) dx = ∫ t·v ds
a = inner(sigma(u), eps(v)) * dx
L = dot(traction, v) * ds(1)  # Applied only on boundary 1 (right)

# ============================================
# Step 4: Solve linear system
# ============================================

# Problem setup (DOLFINx 0.10 syntax)
# petsc_options: dictionary for PETSc solver options
# petsc_options_prefix: prefix for PETSc options (optional)
problem = LinearProblem(a, L, bcs=[bc_left],
                        petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
                        petsc_options_prefix="elasticity_")

# Solve (now uh contains the actual displacement solution!)
uh = problem.solve()

print(f"✓ Displacement solution computed")
print(f"  Maximum displacement: {np.max(np.abs(uh.x.array)):.6e} m")

# ============================================
# Step 5: Calculate von Mises stress
# ============================================

# Deviatoric stress
sigma_dev = sigma(uh) - (1 / 3) * tr(sigma(uh)) * Identity(len(uh))

# von Mises stress
sigma_vm = sqrt((3 / 2) * inner(sigma_dev, sigma_dev))

# Interpolate to DG0 space
W = fem.functionspace(msh, ("Discontinuous Lagrange", 0))
sigma_vm_expr = fem.Expression(sigma_vm, W.element.interpolation_points)
sigma_vm_h = fem.Function(W)
sigma_vm_h.interpolate(sigma_vm_expr)

print(f"✓ von Mises stress computed")
print(f"  Minimum stress: {np.min(sigma_vm_h.x.array):.6e} Pa")
print(f"  Maximum stress: {np.max(sigma_vm_h.x.array):.6e} Pa")
print(f"  Average stress: {np.mean(sigma_vm_h.x.array):.6e} Pa")

# ============================================
# Step 6: Save results (for ParaView visualization)
# ============================================

with io.VTXWriter(msh.comm, "displacement.bp", [uh], engine="BP4") as vtx:
    vtx.write(0.0)

with io.VTXWriter(msh.comm, "von_mises_stress.bp", [sigma_vm_h], engine="BP4") as vtx:
    vtx.write(0.0)

print("\n✓ Results saved:")
print("  - displacement.bp (displacement field)")
print("  - von_mises_stress.bp (von Mises stress)")
print("\nVisualize with ParaView!")

# ============================================
# Step 7: Safety check
# ============================================

sigma_yield_steel = 250e6  # Steel yield stress (250 MPa)
max_stress = np.max(sigma_vm_h.x.array)
safety_factor = sigma_yield_steel / max_stress

print(f"\n=== Safety Assessment (Steel Reference) ===")
print(f"Maximum von Mises stress: {max_stress/1e6:.2f} MPa")
print(f"Yield stress: {sigma_yield_steel/1e6:.2f} MPa")
print(f"Safety factor: {safety_factor:.2f}")

if safety_factor > 1.0:
    print("✓ Safe: Material will not yield")
else:
    print("✗ Warning: Material may yield!")

✓ Displacement solution computed
  Maximum displacement: 8.980158e-04 m
✓ von Mises stress computed
  Minimum stress: 6.238832e+05 Pa
  Maximum stress: 1.168873e+06 Pa
  Average stress: 8.741793e+05 Pa

✓ Results saved:
  - displacement.bp (displacement field)
  - von_mises_stress.bp (von Mises stress)

Visualize with ParaView!

=== Safety Assessment (Steel Reference) ===
Maximum von Mises stress: 1.17 MPa
Yield stress: 250.00 MPa
Safety factor: 213.88
✓ Safe: Material will not yield
